In [3]:
def print_board(board):
    """Helper function to display the board"""
    for i in range(3):
        row = ""
        for j in range(3):
            if board[i][j] == 'X':
                row += "X"
            elif board[i][j] == 'O':
                row += "O"
            else:
                row += "_"
            if j < 2:
                row += " | "
        print(row)
        if i < 2:
            print("---------")
    print()

def get_empty_positions(board):
    """Find all empty positions on the board"""
    positions = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                positions.append((i, j))
    return positions

def check_winner(board):
    """Check if someone has won the game"""
    # Check rows
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != ' ':
            return board[i][0]

    # Check columns
    for j in range(3):
        if board[0][j] == board[1][j] == board[2][j] != ' ':
            return board[0][j]

    # Check diagonals
    if board[0][0] == board[1][1] == board[2][2] != ' ':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != ' ':
        return board[0][2]

    return None

def is_game_over(board):
    """Check if game is finished"""
    if check_winner(board) is not None:
        return True
    # Check if board is full
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                return False
    return True

def count_lines_with_n_pieces(board, player, n):
    """Count lines with exactly n pieces of player and no opponent pieces"""
    count = 0
    opponent = 'O' if player == 'X' else 'X'

    # Check all 8 lines (3 rows, 3 columns, 2 diagonals)
    lines = []

    # Add rows
    for i in range(3):
        lines.append([board[i][0], board[i][1], board[i][2]])

    # Add columns
    for j in range(3):
        lines.append([board[0][j], board[1][j], board[2][j]])

    # Add diagonals
    lines.append([board[0][0], board[1][1], board[2][2]])
    lines.append([board[0][2], board[1][1], board[2][0]])

    # Count lines with exactly n player pieces and no opponent pieces
    for line in lines:
        player_count = line.count(player)
        opponent_count = line.count(opponent)
        if player_count == n and opponent_count == 0:
            count += 1

    return count

def evaluate_board(board):
    """Evaluation function from the assignment"""
    # Check for terminal states first
    winner = check_winner(board)
    if winner == 'X':
        return 1000  # X wins
    elif winner == 'O':
        return -1000  # O wins

    # Calculate evaluation using the given formula

    X3 = count_lines_with_n_pieces(board, 'X', 3)
    X2 = count_lines_with_n_pieces(board, 'X', 2)
    X1 = count_lines_with_n_pieces(board, 'X', 1)

    O3 = count_lines_with_n_pieces(board, 'O', 3)
    O2 = count_lines_with_n_pieces(board, 'O', 2)
    O1 = count_lines_with_n_pieces(board, 'O', 1)

    evaluation = (8*X3 + 3*X2 + X1) - (8*O3 + 3*O2 + O1)
    return evaluation

def minimax(board, depth, is_maximizing):
    """Basic minimax algorithm"""
    global minimax_nodes_visited
    minimax_nodes_visited += 1

    # Base case - terminal node
    if is_game_over(board):
        return evaluate_board(board)

    if is_maximizing:  # X's turn (MAX player)
        best_value = float('-inf')
        empty_positions = get_empty_positions(board)

        for pos in empty_positions:
            i, j = pos
            board[i][j] = 'X'  # Make move
            value = minimax(board, depth + 1, False)
            board[i][j] = ' '  # Undo move
            best_value = max(best_value, value)

        return best_value

    else:  # O's turn (MIN player)
        best_value = float('inf')
        empty_positions = get_empty_positions(board)

        for pos in empty_positions:
            i, j = pos
            board[i][j] = 'O'  # Make move
            value = minimax(board, depth + 1, True)
            board[i][j] = ' '  # Undo move
            best_value = min(best_value, value)

        return best_value

def minimax_with_move(board):
    """Find best move using minimax"""
    best_value = float('-inf')
    best_move = None
    empty_positions = get_empty_positions(board)

    print("Evaluating possible moves for X:")
    for pos in empty_positions:
        i, j = pos
        board[i][j] = 'X'  # Try this move
        value = minimax(board, 0, False)  # Get minimax value
        board[i][j] = ' '  # Undo move

        print(f"Move ({i},{j}): Value = {value}")

        if value > best_value:
            best_value = value
            best_move = pos

    return best_move, best_value

def alpha_beta(board, depth, alpha, beta, is_maximizing):
    """Minimax with alpha-beta pruning"""
    global alpha_beta_nodes_visited, pruning_count
    alpha_beta_nodes_visited += 1

    # Base case - terminal node
    if is_game_over(board):
        return evaluate_board(board)

    if is_maximizing:  # X's turn (MAX player)
        best_value = float('-inf')
        empty_positions = get_empty_positions(board)

        for pos in empty_positions:
            i, j = pos
            board[i][j] = 'X'  # Make move
            value = alpha_beta(board, depth + 1, alpha, beta, False)
            board[i][j] = ' '  # Undo move

            best_value = max(best_value, value)
            alpha = max(alpha, best_value)

            if beta <= alpha:
                pruning_count += 1
                break  # Beta cutoff (pruning)

        return best_value

    else:  # O's turn (MIN player)
        best_value = float('inf')
        empty_positions = get_empty_positions(board)

        for pos in empty_positions:
            i, j = pos
            board[i][j] = 'O'  # Make move
            value = alpha_beta(board, depth + 1, alpha, beta, True)
            board[i][j] = ' '  # Undo move

            best_value = min(best_value, value)
            beta = min(beta, best_value)

            if beta <= alpha:
                pruning_count += 1
                break  # Alpha cutoff (pruning)

        return best_value

def alpha_beta_with_move(board):
    """Find best move using alpha-beta pruning"""
    best_value = float('-inf')
    best_move = None
    empty_positions = get_empty_positions(board)
    alpha = float('-inf')
    beta = float('inf')

    print("Evaluating possible moves for X with Alpha-Beta:")
    for pos in empty_positions:
        i, j = pos
        board[i][j] = 'X'  # Try this move
        value = alpha_beta(board, 0, alpha, beta, False)
        board[i][j] = ' '  # Undo move

        print(f"Move ({i},{j}): Value = {value}")

        if value > best_value:
            best_value = value
            best_move = pos

    return best_move, best_value

# Main solution
def solution():
    """Solve the tic-tac-toe assignment"""
    global minimax_nodes_visited, alpha_beta_nodes_visited, pruning_count

    board = [
        [' ', 'X', ' '],  # Row 0
        ['O', ' ', ' '],  # Row 1
        ['X', ' ', 'O']   # Row 2
    ]

    print("=== TIC-TAC-TOE ASSIGNMENT SOLUTION ===")
    print("\nInitial Board State:")
    print_board(board)

    print(f"Initial evaluation: {evaluate_board(board)}")

    # Reset counters
    minimax_nodes_visited = 0
    alpha_beta_nodes_visited = 0
    pruning_count = 0

    print("\n--- MINIMAX ALGORITHM ---")
    best_move_minimax, best_value_minimax = minimax_with_move(board)
    print(f"\nMinimax Result:")
    print(f"Best move for X: Row {best_move_minimax[0]}, Column {best_move_minimax[1]}")
    print(f"Best value: {best_value_minimax}")
    print(f"Nodes visited: {minimax_nodes_visited}")

    print("\n--- ALPHA-BETA PRUNING ALGORITHM ---")
    best_move_ab, best_value_ab = alpha_beta_with_move(board)
    print(f"\nAlpha-Beta Result:")
    print(f"Best move for X: Row {best_move_ab[0]}, Column {best_move_ab[1]}")
    print(f"Best value: {best_value_ab}")
    print(f"Nodes visited: {alpha_beta_nodes_visited}")
    print(f"Pruning occurred: {pruning_count} times")

    print(f"\nEfficiency: Alpha-beta visited {minimax_nodes_visited - alpha_beta_nodes_visited} fewer nodes")
    if minimax_nodes_visited > 0:
        savings = ((minimax_nodes_visited - alpha_beta_nodes_visited) / minimax_nodes_visited) * 100
        print(f"Space savings: {savings:.1f}%")

    # Show final board with optimal move
    print(f"\nBoard after optimal move ({best_move_minimax[0]}, {best_move_minimax[1]}):")
    board[best_move_minimax[0]][best_move_minimax[1]] = 'X'
    print_board(board)

# Global counters for analysis
minimax_nodes_visited = 0
alpha_beta_nodes_visited = 0
pruning_count = 0

# Run the solution
if __name__ == "__main__":
    solution()

=== TIC-TAC-TOE ASSIGNMENT SOLUTION ===

Initial Board State:
_ | X | _
---------
O | _ | _
---------
X | _ | O

Initial evaluation: 0

--- MINIMAX ALGORITHM ---
Evaluating possible moves for X:
Move (0,0): Value = 0
Move (0,2): Value = 1000
Move (1,1): Value = 1000
Move (1,2): Value = 0
Move (2,1): Value = -1000

Minimax Result:
Best move for X: Row 0, Column 2
Best value: 1000
Nodes visited: 225

--- ALPHA-BETA PRUNING ALGORITHM ---
Evaluating possible moves for X with Alpha-Beta:
Move (0,0): Value = 0
Move (0,2): Value = 1000
Move (1,1): Value = 1000
Move (1,2): Value = 0
Move (2,1): Value = -1000

Alpha-Beta Result:
Best move for X: Row 0, Column 2
Best value: 1000
Nodes visited: 135
Pruning occurred: 39 times

Efficiency: Alpha-beta visited 90 fewer nodes
Space savings: 40.0%

Board after optimal move (0, 2):
_ | X | X
---------
O | _ | _
---------
X | _ | O

